<a href="https://colab.research.google.com/github/seongmin0219/AlgorithmDataStructure/blob/master/baseline_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gc
import pandas as pd
import numpy as np
import gc 
from datetime import datetime
import os
import time
import json
# 아이템 연관도를 봐야되는데
import re
from pathlib import Path
import numba as nb
from tqdm import tqdm
from google.colab import drive
import os
from pathlib import Path
import json
import heapq

In [2]:
drive.mount('/gdrive')

Path('./gdrive/MyDrive/')
file_path = '/gdrive/MyDrive/'

if os.path.exists(file_path):
  print('exists')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
exists


In [3]:
DATA_DIR = '/gdrive/MyDrive/data/'

In [4]:
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)
    return json_obj

In [5]:
# Opening JSON file
f = open(DATA_DIR + 'train_user_seq_log.json')

# returns JSON object as
# a dictionary
data = json.load(f)

# Iterating through the jsoa

In [6]:
df = pd.DataFrame(list(data.items()),columns = ['user_id','item_timestamp'])

NameError: ignored

68

In [7]:
def generate_train(df):
    user_ids = []
    item_ids = []
    timestamps = []
    for rows in tqdm(df.iterrows()):
        item_timestamp = rows[1]['item_timestamp']
        user_id = rows[1]['user_id']
        for idx in range(len(item_timestamp)):
            item = item_timestamp[idx][0]
            timestamp = item_timestamp[idx][1]
            user_ids.append(user_id)
            item_ids.append(item)
            timestamps.append(timestamp)
    train_df = pd.DataFrame(user_ids,columns=['user_ids'])
    train_df['item_ids'] = item_ids
    train_df['timestamp'] = timestamps
    return train_df

In [8]:
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)
    return json_obj

In [9]:
train_db = generate_train(df)

672116it [00:35, 19085.99it/s]


In [10]:
train_db = train_db.rename(columns={'item_ids': 'item_id', 'user_ids':'user_id'})

In [11]:
@nb.jit(nopython=True, cache=True)
def get_topk(cnts, topk , k ):
    for item_id , cnt in cnts.items():
        topk[item_id] = np.array(heap_topk(cnt,1,k))

In [13]:
@nb.jit(nopython=True , cache=True)
def heap_topk(cnt, overwrite ,cap):
    q = [(0.0, 0, 0) for _ in range(0)]
    for i, (k,n) in enumerate(cnt.items()):
        if overwrite == 1:
            heapq.heappush(q, (n,i,k))
        else:
            heapq.heappush(q, (n,-i,k))
        if len(q) > cap:
            heapq.heappop(q)
    return [heapq.heappop(q)[2] for _ in range(len(q))][::-1]

In [14]:
topk = nb.typed.Dict.empty(
    key_type=nb.types.int64,
    value_type=nb.types.int64[:])
cnt = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)
cnts = nb.typed.Dict.empty(
    key_type=nb.types.int64,
    value_type=nb.typeof(nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)))

In [15]:
train_db['item_id'] = train_db['item_id'].apply(lambda x: np.int32(x.split("_")[-1]) )
train_db['timestamp'] = train_db['timestamp'].apply(lambda x: time.mktime(datetime.strptime(x,"%Y-%m-%d %H:%M:%S").timetuple()) )

In [16]:
# item similarity 로 가중치 줘도 되겠다 time weighted 와
items_timestamps = train_db[['item_id','timestamp']].values

In [17]:
df['length'] = df['item_timestamp'].apply(lambda x : len(x))
df['idx'] = df['length'].cumsum()
df['idx'] = df['idx'].shift(1)
df.iloc[0]['idx'] = 0
df.fillna(0,inplace=True)
df['idx'] = df['idx'].astype(np.int64)
df['user_id'] = df['user_id'].apply(lambda x : np.int32(x.split('_')[-1]))

/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [18]:
@nb.jit(nopython = True, parallel = True, cache = True)
def get_pairs(items_timestamps,row, cnts,par_n):
    pairs = [{(0, 0): 0.0 for _ in range(0)} for _ in range(par_n)] 

    for par_i in nb.prange(par_n):
      # if par_i % 100 ==0:
        # print(par_i)
      user_id,idx, length = row[par_i]
      get_single_pair(pairs[par_i], items_timestamps ,idx, length)
      
    for par_i in range(par_n):
        for (aid1, aid2), w in pairs[par_i].items():
            if aid1 not in cnts: cnts[aid1] = {0: 0.0 for _ in range(0)}
            cnt = cnts[aid1]
            if aid2 not in cnt: cnt[aid2] = 0.0
            cnt[aid2] += w

    # for par_i in range(par_n):
    #     for (aid1, aid2), w in pairs[par_i].items():
    #         if aid1 not in cnts: cnts[aid1] = {0: 0.0 for _ in range(0)}
    #         cnt = cnts[aid1]
    #         if aid2 not in cnt: cnt[aid2] = 0.0
    #         cnt[aid2] += w

# 어떻게 끊느냐도 중요한데 유저 전체를 보고 timestamp 로 끊는다?
# 시퀀스 학습시키는게
@nb.jit(nopython=True , cache=True)
def get_single_pair(pairs, item_timestamps,idx,length):
    TIME_WEIGHT =1
    mode = TIME_WEIGHT
    min_idx = idx
    max_idx = idx+length

    for i in range(min_idx , max_idx):
        for j in range(i+1, max_idx):
            timestamp_i =  item_timestamps[i][1]
            timestamp_j =  item_timestamps[j][1]
            # 하루가 넘어가면 break
            time_diff = timestamp_j - timestamp_i
            if time_diff  >= 24 * 60 * 60: break
            if item_timestamps[i][0] == item_timestamps[j][0]: continue # 두 아이템이 같으면 같은 세션이라도 넘어간다
            # 가까운 시간이라도 전혀 다른 아이템일 수 있다
            if mode == TIME_WEIGHT:
               #나의 임의 시간 weighted
                if 0 <= time_diff <= 300: w = 5
                elif 300< time_diff <= 600: w = 4
                elif 600< time_diff <=1800: w = 3
                elif 1800 < time_diff <=3600: w = 2
                elif 3600 < time_diff <= 3600 * 6: w = 1
                else: w = 0
              # 정규화해서 weight를 적용하는건가
            # start_time은 session이기 때문이고 이건 user 별로 있기 때문에 session으로는 나누지 않는다 , 전체 기간 대비 얼마나 짧은 기간내에 검색으로 이어졌는지로 판단
# w1 = 1 + 3 * (ts[i] + start_time - 1659304800) / (1662328791 - 1659304800)
#                w2 = 1 + 3 * (ts[j] + start_time - 1659304800) / (1662328791 - 1659304800) 각 세션
            pairs[(item_timestamps[i][0], item_timestamps[j][0])] = w
            pairs[(item_timestamps[j][0], item_timestamps[i][0])] = w


In [19]:
start_time = df.iloc[0]['item_timestamp'][0][1]
end_time = df.iloc[0]['item_timestamp'][-1][1]

In [24]:
#test_user_seq_log['timestamp'] = test_user_seq_log['item_timestamp'].apply(lambda x: np.array([ int(time.mktime(datetime.strptime(val[1],"%Y-%m-%d %H:%M:%S").timetuple())) for val in x ]) )

NameError: ignored

In [ ]:

#start_time_s = time.mktime(datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S").timetuple())

In [ ]:

#end_time_s = time.mktime(datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S").timetuple())

In [ ]:
#ts_i =time.mktime(datetime.strptime( df.iloc[500]['item_timestamp'][0][1], "%Y-%m-%d %H:%M:%S").timetuple())

In [ ]:

#datetime.fromtimestamp(1662328791)

In [1]:

#datetime.fromtimestamp(1659304800)

NameError: ignored

In [118]:
#(ts_i + start_time_s - end_time_s )/ (end_time_s - start_time_s)

212.60009992420274

In [115]:
#(ts_i + start_time_s - end_time_s )/ (end_time_s - start_time_s)

212.60004973698722

In [117]:
# 이러니까 너무 작은 차이가 난다 
#(ts_i + start_time_s - end_time_s )/ (start_times - start_time_s)



212.60009992420274

In [75]:
datetime.fromtimestamp(items_timestamps[1][1])

datetime.datetime(2021, 2, 11, 13, 3, 52)

In [79]:
start_time = items_timestamps[0][1]

In [82]:
# 총 시간 간격으로 나누어주면 얼마나 거기서 많은 비율과 값을 차지하는 지로 가중치를 계산하는 듯하다
(items_timestamps[2][1]  + start_time -1659304800) / (1662328791 - 1659304800)


518.1207447376662

In [77]:
datetime.fromtimestamp(1659304800)  

datetime.datetime(2022, 7, 31, 22, 0)

In [78]:
datetime.fromtimestamp(1662328791)

datetime.datetime(2022, 9, 4, 21, 59, 51)

In [74]:
# datetime.strptime(,"%Y-%m-%d %H:%M:%S").timetuple() 

TypeError: ignored

In [69]:
items_timestamps[0][1]

1613048622.0

In [72]:
items_timestamps[2][1]


1613048647.0

In [20]:
%%time
# 30초에 1프로
# 3000 초 100프로 50분? 갈수록 빨라지는 듯 cache 해서 그런지  1분에 65건 정도니까원래 코드랑비슷하다 내가 이전에 짠거랑 비교하기 작은 차이라도 큰 차이난다 속도가 엄청나게 !!

# cnt저장이 오래걸린다 그래도 할만한 속도로 대폭 줄음
# 빠르고 병렬적인 base line end-to-end를 위ㅎ 
tail = 30
min_idx = 0
# 5개 전까지 확인
length =20
idx = 0
TIME_WEIGHT =1
parallel =1024
# 아니면 top40를 뽑고 거기서 rerank로 섞어주는 게 더 나을 수도 있다
topn = 20
# for start_idx in range(len(grp_df)-5):
n = 200000  #chunk row size
list_df = [train_db[i:i+n] for i in range(0,train_db.shape[0],n)]
chunks_1 = list_df[0]
grp_chunk = chunks_1.groupby('user_id')
# chunk에서 user_id가 짤렸다면 unique user id 보다 길이가 더 늘어날 수 있으므로 len(df)+알파
# 161개의 청크이므로 중복은 161개 정도에서만 발생, 안 발생하도록 코딩하는 게 제일 좋지만
pairs = [{(0,0) : 0.0 for _ in range(0)} for _ in range(len(df)+300)]
total_len = len(df) 
max_idx = len(df)
cnts = nb.typed.Dict.empty(
        key_type = nb.types.int64,
        value_type = nb.typeof(nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)))
# 확연히 다른 속도.. 일단은 자자 60만행에대해서 각 행 처리한듯하니 옛날같았으면 느리네하고 기다릴텐데

for idx in tqdm(range(0,max_idx, parallel)):
    row = df.iloc[idx:min(idx + parallel, max_idx)][['user_id', 'idx', 'length']].values
    par_n = len(row)
    get_pairs(items_timestamps, row, cnts,par_n)


# get topk from counter
topk = nb.typed.Dict.empty(
        key_type = nb.types.int64,
        value_type = nb.types.int64[:])
get_topk(cnts, topk, topn)



  0%|          | 0/657 [00:00<?, ?it/s]<ipython-input-18-dc6fe9c35c6b>:9: NumbaTypeSafetyWarning: unsafe cast from UniTuple(float64 x 2) to UniTuple(int64 x 2). Precision may be lost.
  get_single_pair(pairs[par_i], items_timestamps ,idx, length)
100%|██████████| 657/657 [06:07<00:00,  1.79it/s]


CPU times: user 8min 38s, sys: 6.8 s, total: 8min 45s
Wall time: 6min 30s


In [67]:
del cnts; gc.collect() 

0

In [21]:
import numpy as np


def ndcg(recs: dict, k: int):
    '''
    :param recs: reco result and label
        example:
            {user: {'recs': [reco_item1, ..., reco_item@], 'gt': gt_item(== label)}}
    :param k: ndcg@k
    :return: ndcg@k score
    '''

    ndcg_ = 0
    for user, each in recs.items():
        reco_items = each['recs']
        gt_item = each['gt']

        for idx, reco_item in enumerate(reco_items):
            if gt_item == reco_item:
                if idx < k:
                    ndcg_ += 1 / np.log2(idx + 2)
                break

    return float(ndcg_ / len(recs))


def hit(recs: dict, k: int):
    '''
    :param recs: reco result and label
        example:
            {user: {'recs': [reco_item1, ..., reco_item@], 'gt': gt_item(== label)}}
    :param k: hit@k
    :return: hit@k(==recall@k) score
    ''' 
    hit_ = 0
    for user, each in recs.items():
        reco_items = each['recs']
        gt_item = each['gt']

        for idx, reco_item in enumerate(reco_items):
            if gt_item == reco_item:
                if idx < k:
                    hit_ += 1
                break

    return float(hit_ / len(recs))


In [22]:
#cnts,topk 여기 실행할때 pairs랑 많이 잡아먹나?

test_user_label = load_json('/gdrive/MyDrive/data/test_user_label.json')
test_user_seq_log = load_json('/gdrive/MyDrive/data/test_user_seq_log.json')
test_user_label_df = pd.DataFrame(list(test_user_label.items()), columns=['user_id', 'item_tiemstamp'])
test_user_seq_log = pd.DataFrame(list(test_user_seq_log.items()), columns=['user_id', 'item_timestamp'])
test_user_label_df.head()
test_user_seq_log = test_user_seq_log.rename(columns={'text':'item_timestamp'})

In [23]:
@nb.jit(nopython=True, cache=True)
def inference(row, result_clicks,topk,seq_weight):
  for user_id, item_id, timestamp in row:
    unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type= nb.types.float64) 
    cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)

    # 최신순부터 거꾸로 배열
    new_item_id = item_id[::-1]
    new_timestamp = timestamp[::-1]

    for a in item_id:
    # t = a[0].split("_")[-1]
    # t = int(a[0])
      unique_aids[a] = 0 
    
    if len(unique_aids) >= 20:
      sequence_weight = np.power(2 , np.linspace(seq_weight, 1, len(new_item_id)))[::-1]-1
      for a , w in zip(new_item_id,sequence_weight):
        if a not in cnt:
         cnt[a] = 0
        cnt[a[0]] += w # * 클릭 ,장바구니, 구매에 따른 가중치 추가가능
      result_candidates = heap_topk(cnt, 0, 20)
    else:
      result_candidates = list(unique_aids)
      for a in result_candidates:
        if a not in topk: continue
        for b in topk[a]:
          if b in unique_aids: continue
          if b not in cnt: cnt[b] = 0
          cnt[b] += 1 # 정렬된 topk에서 co-visitation이 높은 아이템을 순서대로 가져와 모자란 길이만큼 채운다
      result_candidates.extend(heap_topk(cnt, 0, 20 - len(result_candidates)))
    result_clicks[user_id] = np.array(result_candidates)

In [23]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [24]:
test_user_seq_log['item_id'] = test_user_seq_log['item_timestamp'].apply(lambda x: np.array([ np.int64(val[0].split("_")[-1]) for val in x])  )
test_user_seq_log['timestamp'] = test_user_seq_log['item_timestamp'].apply(lambda x: np.array([ int(time.mktime(datetime.strptime(val[1],"%Y-%m-%d %H:%M:%S").timetuple())) for val in x ]) )

In [33]:
test_user_seq_log

,user_id,item_timestamp,item_id,timestamp,length,idx
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294...",20,0
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848...",20,20
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680...",20,40
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918...",20,60
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357...",20,80
...,...,...,...,...,...,...
587481,user_829386,"[[item_554650, 2021-05-12 12:55:35], [item_810...","[554650, 810147, 813829, 893014, 130000]","[1620824135, 1620824148, 1620824177, 162082420...",5,7025309
587482,user_829390,"[[item_310178, 2021-05-12 12:56:18], [item_866...","[310178, 866254, 945164, 47147, 851941, 712124...","[1620824178, 1620824190, 1620824223, 162082423...",8,7025314
587483,user_829393,"[[item_115233, 2021-05-12 12:56:42], [item_492...","[115233, 492271]","[1620824202, 1620824207]",2,7025322
587484,user_829397,"[[item_503200, 2021-05-12 12:58:08], [item_517...","[503200, 517483]","[1620824288, 1620824438]",2,7025324


In [34]:
type(test_user_seq_log['item_id'].values[0][0])

numpy.int64

In [35]:
type(test_user_seq_log['timestamp'].values[0][0])


numpy.int64

In [36]:
def generate_train(df):
    user_ids = []
    item_ids = []
    timestamps = []
    for rows in tqdm(df.iterrows()):
        item_timestamp = rows[1]['item_timestamp']
        user_id = rows[1]['user_id']
        for idx in range(len(item_timestamp)):
            item = item_timestamp[idx][0]
            timestamp = item_timestamp[idx][1]
            user_ids.append(user_id)
            item_ids.append(item)
            timestamps.append(timestamp)
    train_df = pd.DataFrame(user_ids,columns=['user_ids'])
    train_df['item_ids'] = item_ids
    train_df['timestamps'] = timestamps
    return train_df

In [37]:

test_db = generate_train(test_user_seq_log)

587486it [00:25, 22613.50it/s]


In [38]:
test_user_seq_log.iloc[0]['idx'] = 0 

/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [39]:
test_user_seq_log['length'] = test_user_seq_log['item_timestamp'].apply(lambda x : len(x))

test_user_seq_log['idx'] = test_user_seq_log['length'].cumsum()
test_user_seq_log['idx'] = test_user_seq_log['idx'].shift(1)
test_user_seq_log.fillna(0,inplace=True)
test_user_seq_log['idx'] = test_user_seq_log['idx'].astype(np.int64) 



In [40]:
test_user_seq_log

,user_id,item_timestamp,item_id,timestamp,length,idx
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294...",20,0
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848...",20,20
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680...",20,40
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918...",20,60
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357...",20,80
...,...,...,...,...,...,...
587481,user_829386,"[[item_554650, 2021-05-12 12:55:35], [item_810...","[554650, 810147, 813829, 893014, 130000]","[1620824135, 1620824148, 1620824177, 162082420...",5,7025309
587482,user_829390,"[[item_310178, 2021-05-12 12:56:18], [item_866...","[310178, 866254, 945164, 47147, 851941, 712124...","[1620824178, 1620824190, 1620824223, 162082423...",8,7025314
587483,user_829393,"[[item_115233, 2021-05-12 12:56:42], [item_492...","[115233, 492271]","[1620824202, 1620824207]",2,7025322
587484,user_829397,"[[item_503200, 2021-05-12 12:58:08], [item_517...","[503200, 517483]","[1620824288, 1620824438]",2,7025324


In [41]:
type(test_user_seq_log['user_id'].values[0])


str

In [42]:
test_user_seq_log

,user_id,item_timestamp,item_id,timestamp,length,idx
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294...",20,0
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848...",20,20
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680...",20,40
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918...",20,60
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357...",20,80
...,...,...,...,...,...,...
587481,user_829386,"[[item_554650, 2021-05-12 12:55:35], [item_810...","[554650, 810147, 813829, 893014, 130000]","[1620824135, 1620824148, 1620824177, 162082420...",5,7025309
587482,user_829390,"[[item_310178, 2021-05-12 12:56:18], [item_866...","[310178, 866254, 945164, 47147, 851941, 712124...","[1620824178, 1620824190, 1620824223, 162082423...",8,7025314
587483,user_829393,"[[item_115233, 2021-05-12 12:56:42], [item_492...","[115233, 492271]","[1620824202, 1620824207]",2,7025322
587484,user_829397,"[[item_503200, 2021-05-12 12:58:08], [item_517...","[503200, 517483]","[1620824288, 1620824438]",2,7025324


In [43]:
test_db = test_db.rename(columns={'user_ids': 'user_id','item_ids':'item_id','timestamps':'timestamp'})

In [44]:
test_db['user_id'] = test_db['user_id'].apply(lambda x : np.int64(x.split("_")[-1]))

In [45]:
test_db['timestamp'] = test_db['timestamp'].apply(lambda x: np.int64(time.mktime(datetime.strptime(x,"%Y-%m-%d %H:%M:%S").timetuple()) ))

In [46]:
test_db['item_id'] = test_db['item_id'].apply(lambda x : np.int64(x.split("_")[-1]))

In [47]:
test_user_seq_log['user_id'] = test_user_seq_log['user_id'].apply(lambda x : np.int64(x.split("_")[-1]))

In [48]:
test_items_timestamps =  test_db[['item_id','timestamp']].values

In [49]:
test_user_seq_log

,user_id,item_timestamp,item_id,timestamp,length,idx
0,0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294...",20,0
1,1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848...",20,20
2,2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680...",20,40
3,3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918...",20,60
4,4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357...",20,80
...,...,...,...,...,...,...
587481,829386,"[[item_554650, 2021-05-12 12:55:35], [item_810...","[554650, 810147, 813829, 893014, 130000]","[1620824135, 1620824148, 1620824177, 162082420...",5,7025309
587482,829390,"[[item_310178, 2021-05-12 12:56:18], [item_866...","[310178, 866254, 945164, 47147, 851941, 712124...","[1620824178, 1620824190, 1620824223, 162082423...",8,7025314
587483,829393,"[[item_115233, 2021-05-12 12:56:42], [item_492...","[115233, 492271]","[1620824202, 1620824207]",2,7025322
587484,829397,"[[item_503200, 2021-05-12 12:58:08], [item_517...","[503200, 517483]","[1620824288, 1620824438]",2,7025324


In [50]:
@nb.jit(nopython=True, cache=True)
def inference(row, result_clicks,topk,seq_weight):
  for user_id, idx, length in row:
    unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type= nb.types.float64) 
    cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)

    # 최신순부터 거꾸로 배열
    #new_item_id = item_id[::-1]
    #new_timestamp = timestamp[::-1]
    
    candidates = test_items_timestamps[idx:idx + length][::-1]

    # for a in candidates: 
    #   unique_aids[a[0]] = 0.0
    
    # if len(unique_aids) >= 20:
    #   sequence_weight = np.power(2 , np.linspace(seq_weight, 1, len(candidates)))[::-1]-1
    for a in candidates:
      if a[0] not in cnt:
        cnt[a[0]] = 0.0
      #cnt[a[0]] += w # * 클릭 ,장바구니, 구매에 따른 가중치 추가가능

    result_candidates = heap_topk(cnt, 0, len(candidates))
    # else:
      # result_candidates = list(unique_aids)
      # for a in result_candidates:
      #   if a not in topk: continue
      #   for b in topk[a]:
      #     if b in unique_aids: continue
      #     if b not in cnt: cnt[b] = 0.0
      #     cnt[b] += 1 # 정렬된 topk에서 co-visitation이 높은 아이템을 순서대로 가져와 모자란 길이만큼 채운다
      # result_candidates.extend(heap_topk(cnt, 0, 20 - len(result_candidates)))
    result_clicks[user_id] = np.array(result_candidates)

In [51]:
#result place holder
result_clicks = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])
result_buy = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:]) 

NameError: ignored

In [52]:
for idx in tqdm(range(0, len(test_user_seq_log), parallel)):
  row = test_user_seq_log.iloc[idx:min(idx+parallel , len(test_user_seq_log))][['user_id','idx','length']].values
  inference(row , result_clicks,topk,0.1)

  0%|          | 0/574 [00:00<?, ?it/s]<ipython-input-50-a74ab4cee94a>:1: NumbaWarning: Cannot cache compiled function "inference" as it uses dynamic globals (such as ctypes pointers and large global arrays)
  @nb.jit(nopython=True, cache=True)
100%|██████████| 574/574 [00:02<00:00, 202.77it/s]


In [58]:
test_user_seq_log

,user_id,item_timestamp,item_id,timestamp,length,idx
0,0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294...",20,0
1,1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848...",20,20
2,2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680...",20,40
3,3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918...",20,60
4,4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357...",20,80
...,...,...,...,...,...,...
587481,829386,"[[item_554650, 2021-05-12 12:55:35], [item_810...","[554650, 810147, 813829, 893014, 130000]","[1620824135, 1620824148, 1620824177, 162082420...",5,7025309
587482,829390,"[[item_310178, 2021-05-12 12:56:18], [item_866...","[310178, 866254, 945164, 47147, 851941, 712124...","[1620824178, 1620824190, 1620824223, 162082423...",8,7025314
587483,829393,"[[item_115233, 2021-05-12 12:56:42], [item_492...","[115233, 492271]","[1620824202, 1620824207]",2,7025322
587484,829397,"[[item_503200, 2021-05-12 12:58:08], [item_517...","[503200, 517483]","[1620824288, 1620824438]",2,7025324


In [50]:
result_clicks[829403]

array([567048, 537578, 628847, 335217, 263005, 167515, 292633, 183824])

In [53]:

# 한 아이템에 이 데이터들 저장하는게 5분이면 아주 빠르다 굿 안알려진거라 공유 받은 이거 나만 잘쓰고 있는 걸 수도 드디어 해냈따 자료구조는 컴퓨터과학적으로 생각하고 꼼꼼함을 곁들여서 정성적으로
cnts[710805]

DictType[int64,float64]<iv=None>({205108: 974.0, 911302: 24.0, 27526: 434.0, 561965: 24.0, 716642: 66.0, 247703: 70.0, 180739: 40.0, 775306: 99.0, 852611: 108.0, 267761: 19.0, 753713: 506.0, 35296: 1097.0, 310343: 214.0, 487164: 21.0, 129407: 47.0, 102468: 65.0, 883209: 31.0, 556108: 2.0, 153293: 25.0, 399654: 501.0, 569886: 233.0, 146176: 67.0, 764004: 1382.0, 885499: 1456.0, 396738: 458.0, 319370: 433.0, 490906: 3318.0, 424308: 2440.0, 204302: 54.0, 344325: 5.0, 163988: 713.0, 31473: 284.0, 385955: 183.0, 720743: 461.0, 114789: 1284.0, 268392: 36.0, 364964: 56.0, 383142: 10.0, 731240: 6.0, 23674: 68.0, 819060: 62.0, 229779: 58.0, 53796: 47.0, 26553: 24.0, 381746: 24.0, 108179: 13.0, 366524: 34.0, 426840: 201.0, 825840: 142.0, 19529: 349.0, 466540: 125.0, 716120: 8.0, 743305: 145.0, 931859: 30.0, 379100: 13.0, 910858: 12.0, 35791: 4.0, 810648: 2.0, 363368: 170.0, 199691: 53.0, 194914: 423.0, 88696: 104.0, 479021: 49.0, 371235: 578.0, 416605: 65.0, 298705: 35.0, 32355: 76.0, 937224: 12

In [53]:

item_names_table = load_json('/gdrive/MyDrive/data/item_names_table.json')
item_df = pd.DataFrame(list(item_names_table.items()),columns = ['item_id','text'])
item_df['text_re'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s) for s in item_df['text']]
item_df.memory_usage()

NameError: ignored

In [54]:

bag_of_words  ={}

In [55]:


bag_of_words = {}
def get_bag_of_words(row,bag_of_words):
  for word in row.split(" "):
    if word in bag_of_words.keys():
      continue
    else:
      bag_of_words[word]  = len(bag_of_words)


In [56]:
len(bag_of_words)

0

In [57]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/main 

In [58]:
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.21).
git is already the newest version (1:2.17.1-1ubuntu0.13).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [59]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [60]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp38-cp38-linux_x86_64.whl size=142140 sha256=d9aff078468333b89dfea46613a893d1b487175aaf89cbb61a6fbc9405506d3e
  Stored in directory: /root/.cache/pip/wheels/e3/88/e7/a947778cce3c142d5721c0629e05db7b09979d3a973277ec2f
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [61]:

import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint 
okt = Okt()
sentence = "안녕하세요 저는 천재입니다."
temp_X = okt.nouns(sentence)
temp_X

['저', '천재']

In [93]:
del df

In [94]:
del train_db

In [95]:
gc.collect()

76

In [96]:
for i in item_df['text_re'].values:
  break

In [102]:
bag_of_words['빅사이즈']

2

In [103]:

sentence = i.split(" ")

In [111]:
temp

[]

In [ ]:
token

In [62]:
 
token = []
embeddingmodel = []
size = 0
for i in item_df['text_re'].values:
    size += 1
    sentence = i.split(" ")
    temp = []
    temp_embedding = []
    all_temp = []
    for k in range(len(sentence)):
        if sentence[k] == '':
          continue
        temp_embedding.append(sentence[k])
        temp.append(sentence[k])
    all_temp.append(temp)
    embeddingmodel.append(temp_embedding)
    # category = i[1]  # csv에서 category column으로 변경
    # category_number_dic = {'IT과학': 0, '경제': 1, '정치': 2, '사회': 3, '스포츠': 4, '오피니언': 5, '세계': 6, '한국야구': 4, '해외야구': 4,
    #                        '해외축구': 4, '한국축구': 4, '농구': 4, '배구': 4, '일반 스포츠': 4, 'e스포츠': 4}

    # config 안쓰고 명시하는 것도 괜찮을 듯
    # category_number_dic = config.Config.category_num_dict
    token.append(all_temp)
print("토큰 처리 완료")

토큰 처리 완료


In [63]:
embeddingmodel = []
for i in range(len(token)):
    temp_embeddingmodel = []
    for k in range(len(token[i][0])):
        temp_embeddingmodel.append(token[i][0][k])
    embeddingmodel.append(temp_embeddingmodel)
# 3.8 gensim vesion sg =1 skip-gram54
# word 2 vec hyper parameters 


In [101]:
from gensim.models import Word2Vec,Doc2Vec

In [71]:
gc.collect()

602

In [108]:
from gensim.test.utils import common_texts

In [109]:
common_texts_and_tags = [
    (text, [f"str_{i}",]) for i, text in enumerate(common_texts)
]

In [110]:
print("##"*20)
print("tags and its texts")
print("##"*20)
for text, tags in common_texts_and_tags:
    print(f"tags: {tags}, text: {text}")

########################################
tags and its texts
########################################
tags: ['str_0'], text: ['human', 'interface', 'computer']
tags: ['str_1'], text: ['survey', 'user', 'computer', 'system', 'response', 'time']
tags: ['str_2'], text: ['eps', 'user', 'interface', 'system']
tags: ['str_3'], text: ['system', 'human', 'system', 'eps']
tags: ['str_4'], text: ['user', 'response', 'time']
tags: ['str_5'], text: ['trees']
tags: ['str_6'], text: ['graph', 'trees']
tags: ['str_7'], text: ['graph', 'minors', 'trees']
tags: ['str_8'], text: ['graph', 'minors', 'survey']


In [144]:
item_texts_and_tags = item_df[['text_re','item_id']].values

In [164]:
def remove_whitespace(text):
  result = []
  for value in text:
      if value =='':
        continue
      if value not in result:
          result.append(value)
  return result


In [165]:
item_texts_and_tags_tuning = list(map(lambda x: (remove_whitespace(x[0].split(" ")),[x[1]]) , item_texts_and_tags))

In [127]:
from gensim.models.doc2vec import Doc2Vec

In [104]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

['텐피엠', '카시오', 'MTP', 'V005L', '남성가죽시계', '블랙', '나토밴드', '']

[(['human', 'interface', 'computer'], ['str_0']),
 (['survey', 'user', 'computer', 'system', 'response', 'time'], ['str_1']),
 (['eps', 'user', 'interface', 'system'], ['str_2'])]

In [138]:
[TaggedDocument(words=text, tags=tags) for text, tags in common_texts_and_tags]

[TaggedDocument(words=['human', 'interface', 'computer'], tags=['str_0']),
 TaggedDocument(words=['survey', 'user', 'computer', 'system', 'response', 'time'], tags=['str_1']),
 TaggedDocument(words=['eps', 'user', 'interface', 'system'], tags=['str_2']),
 TaggedDocument(words=['system', 'human', 'system', 'eps'], tags=['str_3']),
 TaggedDocument(words=['user', 'response', 'time'], tags=['str_4']),
 TaggedDocument(words=['trees'], tags=['str_5']),
 TaggedDocument(words=['graph', 'trees'], tags=['str_6']),
 TaggedDocument(words=['graph', 'minors', 'trees'], tags=['str_7']),
 TaggedDocument(words=['graph', 'minors', 'survey'], tags=['str_8'])]

In [167]:

# text: 단어별로 분할되어 있는 리스트 
# tags: 해당 문서를 의미하는 태그. 여기서, tags를 unique id로 넣기는 했는데, 그렇지 않게 넣어서 학습시키는 것도 가능할 것으로 보임. 
TRAIN_documents = [TaggedDocument(words=text, tags=tags) for text, tags in item_texts_and_tags_tuning]

[TaggedDocument(words=['DA', '남자', '빅사이즈', '절개', '맨투맨', '트레이닝팬츠', 'SET'], tags=['item_0']),
 TaggedDocument(words=['텐피엠', '카시오', 'MTP', 'V005L', '남성가죽시계', '블랙', '나토밴드'], tags=['item_1']),
 TaggedDocument(words=['바보사랑', '여성', '옥스퍼드', '로퍼', '주', '태블리스'], tags=['item_2']),
 TaggedDocument(words=['해외', '블랙모어스', '리프리놀', '초록홍합', '100캡슐', 'X', '3'], tags=['item_3']),
 TaggedDocument(words=['백화점', 'SAY', '크로커다일', '경량덕', '다운', '점퍼', 'CL9WDW901'], tags=['item_4'])]

In [169]:
#https://frhyme.github.io/python-libs/nlp_doc2vec_gensim/
#https://radimrehurek.com/gensim/similarities/docsim.html
model = Doc2Vec(TRAIN_documents, vector_size=5, window=3, epochs=5, min_count=1, workers=4)

In [ ]:
model.docvecs['item_0']

In [183]:
vec = model.docvecs['item_0']

In [177]:
inferred_v = model.infer_vector(['DA', '남자', '빅사이즈', '절개', '맨투맨', '트레이닝팬츠', 'SET'])

In [185]:
# 하나도 안비슷한데 흠 뭐가 잘못된듯

item_df[item_df.item_id.isin(['item_0','item_883996','item_54712','item_851321'])]

,item_id,text,text_re
0,item_0,DA 남자 빅사이즈 절개 맨투맨 절개 트레이닝팬츠 SET,DA 남자 빅사이즈 절개 맨투맨 절개 트레이닝팬츠 SET
54712,item_54712,리스트 탈착 타이넥 솔리드 셔츠_TWWSTJ82010,리스트 탈착 타이넥 솔리드 셔츠 TWWSTJ82010
851321,item_851321,리스트 [리스트]프릴 슬리브 스트라이프 코튼 셔츠_TWWSTJ70140,리스트 리스트 프릴 슬리브 스트라이프 코튼 셔츠 TWWSTJ70140
883996,item_883996,[대구백화점 II관][르피타]울 단추 니트 베스트 조끼(L204UVTB51),대구백화점 II관 르피타 울 단추 니트 베스트 조끼 L204UVTB51


In [184]:
 # 오케이 많이 비슷하다, 탈착과 절개도 비슷하다고 판단하네
 # 이렇게 most_similar item을 찾는 건 알았는데 
# 특정 아이템과의 유사도도 계산해보자 ,이름이비슷하다고다음검색으로 이어지진않으니까 그게 벡터화된 값이라 해도 
# 드디어 cls token같은 doc2vec 을 해봤고 해보니까 또 연구하듯이 생각드는건 이 압축된정보가 정보를 더 잘 포함할까? cls token이 그랬떤것처럼 거기에 더 잘 담을려면 어떻게해야되는거고 이렇게 유사도가 잘나오는데 그러면 word token vector는 필요없나

 model.docvecs.most_similar([vec], topn=3)

[('item_0', 1.0),
 ('item_883996', 0.9996147155761719),
 ('item_54712', 0.9981741309165955)]

In [ ]:
#https://hanshuginn.blogspot.com/2019/03/python-doc2vec.html
model.g

In [ ]:

# 코사인유사도

In [ ]:

#https://kmikey1004.tistory.com/2

In [ ]:
# document의 vector를 구하는 방법은, 기 학습된 docvec을 가져오거나, 해당 단어를 넣고 다시 예측하는 것. 
# 단, tag로 접근이 가능할 뿐, 학습한 doc를 그대로 가져오는 것은 어려운 것으로 보임. 
# 또한, 여기서, 학습된 vector와 inferring한 vector가 미묘하게 다른데, 이것은 현재 train set이 적어서 그런 것으로 보임. 
 #역시 이사람이 잘못안거고 model.infer_vector는 없는 문서일 경우에 랜덤으로 벡터값을 초기화하는 메서드이다
 # model.docvecs[{tag_id}] 로 가져와야함
# model.docvecs.most_similar([vec], topn=3) 이게 그증거
# [('item_0', 1.0),
#  ('item_883996', 0.9996147155761719),
#  ('item_54712', 0.9981741309165955)]

In [ ]:
 model.docvecs.most_similar([inferred_v], topn=3)


In [ ]:
for text, tags in common_texts_and_tags:
    trained_doc_vec = model.docvecs[tags[0]]
    inferred_doc_vec = model.infer_vector(text)
    print(f"tags: {tags}, text: {text}")
    print(f"trained_doc_vec: {trained_doc_vec}")
    print(f"inferred_doc_vec: {inferred_doc_vec}")
    print("--"*20)

In [170]:
embedding.save(f"doc2vec.embedding")

NameError: ignored

In [ ]:
# https://frhyme.github.io/python-libs/nlp_doc2vec_gensim/ doc2vec

In [ ]:
for text, tags in common_texts_and_tags:
    trained_doc_vec = model.docvecs[tags[0]]
    inferred_doc_vec = model.infer_vector(text)
    print(f"tags: {tags}, text: {text}")
    print(f"trained_doc_vec: {trained_doc_vec}")
    print(f"inferred_doc_vec: {inferred_doc_vec}")
    print("--"*20)
    
print("##"*20)
print("predicting Document vector")
print("##"*20)

new_documents = [
    ["computer", "interface"], 
    ["I", "am", "a", "boy"]
]
# predict training set with its infering vector)
for text in new_documents:
    inferred_v = model.infer_vector(text)
    # 현재 doc를 모델을 사용하여 벡터화할때의 값 
    print(f"vector of {text}: {inferred_v}")
    # 기학습된 문서중에서 현재 벡터와 가장 유사한 벡터를 가지는 문서를 topn만큼 추출합니다. 
    most_similar_docs = model.docvecs.most_similar([inferred_v], topn=3)
    # index와 그 유사도를 함께 보여줍니다. 
    # index(tag)가 아닌 문서를 바로 보여주기는 어려운 것 같고, 
    for index, similarity in most_similar_docs:
        print(f"{index}, similarity: {similarity}")
    #print(most_similar_docs)
    print("=="*20)


In [66]:
embedding = Word2Vec(embeddingmodel,size=vector_size, window=3, min_count=5, iter=5, sg=1, max_vocab_size=360000000)
# 4.0.1 gensim version
# embedding = Word2Vec(embeddingmodel, vector_size=300, window=5, min_count=10, epochs=5, sg=1, max_vocab_size=360000000)
#embedding.save(f"embedding/{portal_site}_portal_post.embedding")

#
embedding.save(f"product.embedding")

print('word embedding 저장 완료')




word embedding 저장 완료


585

In [78]:
import gensim
model_name = 'product.embedding'

In [79]:
# ram 이 부족하다 아무래도
model = gensim.models.word2vec.Word2Vec.load(model_name)


In [84]:
item_df

,item_id,text,text_re
0,item_0,DA 남자 빅사이즈 절개 맨투맨 절개 트레이닝팬츠 SET,DA 남자 빅사이즈 절개 맨투맨 절개 트레이닝팬츠 SET
1,item_1,텐피엠 [카시오] MTP-V005L 남성가죽시계+ 블랙 나토밴드.,텐피엠 카시오 MTP V005L 남성가죽시계 블랙 나토밴드
2,item_2,[바보사랑]여성 옥스퍼드 로퍼-(주)태블리스,바보사랑 여성 옥스퍼드 로퍼 주 태블리스
3,item_3,[해외][블랙모어스] 리프리놀/ 초록홍합 100캡슐 X 3,해외 블랙모어스 리프리놀 초록홍합 100캡슐 X 3
4,item_4,[백화점 SAY][크로커다일]경량덕 다운 점퍼(CL9WDW901),백화점 SAY 크로커다일 경량덕 다운 점퍼 CL9WDW901
...,...,...,...
970240,item_970240,접이식 벤치의자 스툴 스탠드 의자 홈바 인테리어 카페,접이식 벤치의자 스툴 스탠드 의자 홈바 인테리어 카페
970241,item_970241,[오너클랜]기획 신발정리대 소형 6단 1P 현관신발장 오픈신발장,오너클랜 기획 신발정리대 소형 6단 1P 현관신발장 오픈신발장
970242,item_970242,[피우다][피우다 (PIUDA)] 여름바다 상자 3p 세트 (수납함/수납,피우다 피우다 PIUDA 여름바다 상자 3p 세트 수납함 수납
970243,item_970243,[지큐][지큐/쿨제이][빅사이즈 추가] 모자이크 롱 숄 가디건,지큐 지큐 쿨제이 빅사이즈 추가 모자이크 롱 숄 가디건


In [ ]:

model.wv.most_similar('로퍼')


In [89]:
from gensim.similarities import Similarity
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile

index_tmpfile = get_tmpfile("index")
# query = [(1, 2), (6, 1), (7, 2)]
query = [(0,1), (1,1), (2, 1)]

index = Similarity(index_tmpfile, common_corpus, num_features=len(common_dictionary))  # build the index
similarities = index[query]

/usr/local/lib/python3.8/dist-packages/gensim/similarities/docsim.py:518: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.hstack(shard_results)


In [93]:
common_corpus_2 = [[ 1,2,3,4,5 ], [5,6,7,8,9]]

In [98]:

common_corpus_2 = [(1,2,3,4,5 ), (5,6,7,8,9)]

In [97]:
query= [(1,2,3,4,5)]

In [99]:
index = Similarity(index_tmpfile, common_corpus, num_features=len(common_dictionary))  # build the index
similarities = index[query]

ValueError: ignored

In [90]:
# query 를 보내고, index로 표현해서 similarity 를 보자
similarities

array([0.99999994, 0.23570226, 0.28867513, 0.23570226, 0.        ,
       0.        , 0.        , 0.        , 0.        ], dtype=float32)

In [88]:
common_corpus

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]

In [87]:
similarities

array([0.38490018, 0.4082483 , 0.33333334, 0.27216554, 0.57735026,
       0.        , 0.        , 0.        , 0.        ], dtype=float32)

In [85]:

model.wv.most_similar('로퍼')


[('드라이빙', 0.8247734308242798),
 ('블로퍼', 0.8132685422897339),
 ('드라이빙슈즈', 0.8063546419143677),
 ('페니', 0.7777316570281982),
 ('페니로퍼', 0.7719422578811646),
 ('플랫슈즈', 0.770158052444458),
 ('단화', 0.7659959197044373),
 ('옥스포드화', 0.760817289352417),
 ('여성로퍼', 0.7542648911476135),
 ('moyhea', 0.7501505613327026)]

In [81]:

model.wv.most_similar('치마')


[('주름치마', 0.7885022163391113),
 ('여성치마', 0.7832591533660889),
 ('롱스커트', 0.7804417014122009),
 ('스커트', 0.7798577547073364),
 ('여성스커트', 0.7776222229003906),
 ('롱치마', 0.7768256068229675),
 ('치마반바지', 0.7752245664596558),
 ('랩스커트', 0.774721622467041),
 ('밴드치마', 0.7703984975814819),
 ('전통무용', 0.7697372436523438)]

In [83]:
model.wv.most_similar('섹스')


[('32ml', 0.9487873315811157),
 ('세비엄', 0.9381943941116333),
 ('식물나라', 0.9376633763313293),
 ('윤활제', 0.9373548030853271),
 ('러브젤', 0.9371756315231323),
 ('메디렌즈', 0.9337586164474487),
 ('립테라피', 0.9335440993309021),
 ('흑설탕', 0.9328588247299194),
 ('10mL', 0.9317175149917603),
 ('딥클렌징폼', 0.9315555095672607)]

In [ ]:
model = gensim.models.word2vec.Word2Vec.load(model_name)
    # model = spacy.load(model_name)
    #    for sentence in sentences:
    if len(sentences) == 1:
        for word in sentences:

            word_cnt += 1
            # model.wv.vocab delete -> model.wv.key_to_index
            # model 값에서 단어가 품사와 같이 코딩되어 있는 지 check하기
            if (word in model.wv.key_to_index):
                sub.append(model.wv[word])
                coverage_cnt +=1
            else:
                sub.append(np.random.uniform(-0.25, 0.25, 300))  # used for OOV words
            # if len(sub) > max_seq_length:
            #    max_seq_length = len(sub)
        word_vec.append(sub)
        print(f"coverage rate : {coverage_cnt / word_cnt} ")
        # word_vec.append(sub)
        #    except Exception as e:
        #        print(e)
        # total_batch = len(word_vec)
        total_batch = len(word_vec)
        return word_vec, max_seq_length, total_batch
    else:
        for sentence in sentences:
            sub = []

            for word in sentence:
                word_cnt += 1
                # model.wv.vocab delete -> model.wv.key_to_index
                # model 값에서 단어가 품사와 같이 코딩되어 있는 지 check하기
                if (word in model.wv.key_to_index):
                    sub.append(model.wv[word])
                    coverage_cnt += 1
                else:
                    sub.append(np.random.uniform(-0.25, 0.25, 300))  # used for OOV words
                if len(sub) == 1190:
                    print(len(sub))
            word_vec.append(sub)
        total_batch = len(word_vec)
        print(f"coverage rate : {coverage_cnt / word_cnt} ")
        return word_vec, max_seq_length, total_batch

In [82]:
item_df['text_re'].apply(lambda x : get_bag_of_words(x,bag_of_words))

0         None
1         None
2         None
3         None
4         None
          ... 
970240    None
970241    None
970242    None
970243    None
970244    None
Name: text_re, Length: 970245, dtype: object

In [ ]:
np.linspace(0.1 , 1 , 20)

array([0.1       , 0.14736842, 0.19473684, 0.24210526, 0.28947368,
       0.33684211, 0.38421053, 0.43157895, 0.47894737, 0.52631579,
       0.57368421, 0.62105263, 0.66842105, 0.71578947, 0.76315789,
       0.81052632, 0.85789474, 0.90526316, 0.95263158, 1.        ])

In [ ]:
# sequence_weight
np.power(2, np.linspace(0.1 , 1 , 20))[::-1] -1

array([1.        , 0.93539975, 0.87288609, 0.81239163, 0.75385116,
       0.69720154, 0.64238172, 0.58933258, 0.53799694, 0.48831944,
       0.44024654, 0.39372639, 0.34870885, 0.30514539, 0.26298903,
       0.22219432, 0.18271729, 0.14451537, 0.10754738, 0.07177346])

In [60]:
recs ={}
# del cnts
# del test_user_label
# del topk
# 만들기 전 메모리 관리 
for user in test_user_seq_log['user_id'].values: 
  str_user = 'user_'+str(user)
  recs[str_user] = {'recs' :list(map(lambda x: 'item_'+str(x), result_clicks[user])), 'gt' : test_user_label[str_user][0] }
hit_20  = hit(recs,20)

In [61]:
# no time-weighted only last 20 
# 0.16 , only popularity
hit_20

0.16606863823137913

In [62]:
# no time-weighted only last 20 
# 0.16 , only popularity
hit_10  = hit(recs,10)

In [63]:
hit_10

0.15580796818988027

In [56]:
# 0.24 , baseline metric 의 경우에는 hit_10이 0.19로 많이 떨어지는 편이다
# time weighted 는 
# if 0 <= time_diff <= 300: w = 5
# elif 300< time_diff <= 600: w = 4
# elif 600< time_diff <=1800: w = 3
# elif 1800 < time_diff <=3600: w = 2
# elif 3600 < time_diff <= 3600 * 6: w = 1 이렇게 줬을 때의 결과값

hit_20

0.242431309001406

In [57]:
hit_1  = hit(recs,1)

In [58]:
hit_1

0.0

In [59]:
ndcg_10  = ndcg(recs,10)

In [60]:
ndcg_10

0.09579339900110684